In [1]:

import time
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from IPython.display import display, Math


In [15]:
def get_trans_mat(a, alpha, d, theta, return_P=False):
    """
    Obtain the transformation matrix from DH Parameters
    :param a: Link angle (in deg)
    :param alpha: Link twist
    :param d: Link offset (in deg)
    :param theta: Joint angle
    :param return_P: return the 3x1 translation matrix
    :return: transformation matrix and rotation matrix
    """
    if type(alpha) == int or type(alpha) == float:
        alpha = alpha*pi/180
    if type(theta) == int or type(theta) == float:
        theta = theta*pi/180

    R = rotation(alpha, 'x') * rotation(theta, 'z')
    R.simplify()
    T = add_translation(R, Matrix([a, -sin(alpha) * d, cos(alpha) * d, 1]))
    if return_P:
        return R, T, Matrix([a, -sin(alpha) * d, cos(alpha) * d])
    else:
        return R, T
    

def rotation(theta, direction):
    if direction == 'x':
        R = Matrix([[1, 0, 0],
                    [0, cos(theta), -sin(theta)],
                    [0, sin(theta), cos(theta)]])
    elif direction == 'y':
        R = Matrix([[cos(theta), 0, sin(theta)],
                    [0, 1, 0],
                    [-sin(theta), 0, cos(theta)]])
    elif direction == 'z':
        R = Matrix([[cos(theta), -sin(theta), 0],
                    [sin(theta), cos(theta), 0],
                    [0, 0, 1]])
    return R

def add_translation(R, t=Matrix([0, 0, 0, 1])):
    T = R.row_insert(len(R), Matrix([[0, 0, 0]]))
    T = T.col_insert(len(T), t)
    return T

def symprint(symbol, sup, sub, dot=False):
    if dot == 1:
        symbol = r'\dot{%s}' % symbol
    elif dot == 2:
        symbol = r'\ddot{%s}' % symbol
    if sup == '':
        info = r"{}_{}".format(symbol, sub)
    else:
        info = r"^{}{}_{}".format(sup, symbol, sub)
    display(symbols(info))

def matprint(matrix, alias=None):
    if alias:
        display(matrix.subs(alias))
    else:
        display(matrix)

In [6]:

# Variables
t = symbols('t')
theta1 = Function(r"\theta_2")(t)
theta1_dot = diff(theta1, t)
theta1_ddot = diff(theta1, t, t)

d2 = Function('d_2')(t)
d2_dot = diff(d2, t)
d2_ddot = diff(d2, t, t)

d3 = Function('d_3')(t)
d3_dot = diff(d3, t)
d3_ddot = diff(d3, t, t)

theta4 = Function(r"\theta_4")(t)
theta4_dot = diff(theta4, t)
theta4_ddot = diff(theta4, t, t)


alias = {}
# display as x dot instead of dx/dt, and ignore dependency (t)
alias.update({theta1: symbols(r"\theta_1"),
              theta1_dot: symbols(r"\dot{\theta_1}"),
              theta1_ddot: symbols(r"\ddot{\theta_1}"),
              theta4: symbols(r"\theta_4"),
              theta4_dot: symbols(r"\dot{\theta_4}"),
              theta4_ddot: symbols(r"\ddot{\theta_4}"),
              d2: symbols(r"d_2"),
              d2_dot: symbols(r"\dot{d_2}"),
              d2_ddot: symbols(r"\ddot{d_2}"),
              d3: symbols(r"d_3"),
              d3_dot: symbols(r"\dot{d_3}"),
              d3_ddot: symbols(r"\ddot{d_3}"),}) 

# display cos(theta) as c\theta, sin(theta) as s\theta
alias.update({sin(theta1): symbols(r"s\theta_1"),
              cos(theta1): symbols(r"c\theta_1"),
              sin(theta4): symbols(r"s\theta_4"),
              cos(theta4): symbols(r"c\theta_4")})



In [11]:
R01, T01, P01 = get_trans_mat(0, 0, 0, theta1, return_P=True)
R12, T12, P12 = get_trans_mat(0, 0, 20+d2, 0, return_P=True)
R23, T23, P23 = get_trans_mat(0, 90, d3, 0, return_P=True)
R34, T34, P34 = get_trans_mat(0, 0, 15, theta4, return_P=True)
R4e, T4e, P4e = get_trans_mat(1, 0, 4, 0, return_P=True)

R10 = R01.T
R21 = R12.T
R32 = R23.T
R43 = R34.T
Re4 = R4e.T

R0e = R01 * R12 * R23 * R34 * R4e
T0e = T01 * T12 * T23 * T34 * T4e

In [17]:
print('Frame 0 (Base)')
W00 = zeros(3, 1)
symprint('\Omega', 0, 0)
matprint(W00, alias)
V00 = zeros(3, 1)
symprint('V', 0, 0)
matprint(V00, alias)

print('Frame 1')
W11 = R10 * W00 + Matrix([0, 0, theta1_dot])
symprint('\Omega', 1, 1)
matprint(W11, alias)
V11 = R10 * (V00 + W00.cross(P01))
symprint('V', 1, 1)
matprint(V11, alias)

print('Frame 2')
W22 = R21 * W11
symprint('\Omega', 2, 2)
matprint(W22, alias)
V22 = R21 * (V11 + W11.cross(P12)) + Matrix([0, 0, d2_dot])
symprint('V', 2, 2)
matprint(V22, alias)

print('Frame 3')
W33 = R32 * W22
symprint('\Omega', 3, 3)
matprint(W33, alias)
V33 = R32 * (V22+ W22.cross(P23)) + Matrix([0, 0, d3_dot])
symprint('V', 3, 3)
matprint(V33, alias)

print('Frame 4')
W44 = R43 * W33 + Matrix([0, 0, theta4_dot])
W44.simplify()
symprint('\Omega', 4, 4)
matprint(W44, alias)
V44 = R43 * (V33 + W33.cross(P34))
V44.simplify()
symprint('V', 4, 4)
matprint(V44, alias)

print('Frame e (end effector)')
Wee = Re4 * W44
Wee.simplify()
symprint('\Omega', 'e', 'e')
matprint(Wee, alias)
Vee = Re4 * (V44 + W44.cross(P4e))
Vee.simplify()
symprint('V', 'e', 'e')
matprint(Vee, alias)

Frame 0 (Base)


^0\Omega_0

Matrix([
[0],
[0],
[0]])

^0V_0

Matrix([
[0],
[0],
[0]])

...

Frame e (end effector)


^e\Omega_e

Matrix([
[\dot{\theta_1}*s\theta_4],
[\dot{\theta_1}*c\theta_4],
[          \dot{\theta_4}]])

^eV_e

Matrix([
[                  \dot{\theta_1}*c\theta_4*d_3 + 19*\dot{\theta_1}*c\theta_4 + \dot{d_2}*s\theta_4],
[-\dot{\theta_1}*d_3*s\theta_4 - 19*\dot{\theta_1}*s\theta_4 + \dot{\theta_4} + \dot{d_2}*c\theta_4],
[                                                             -\dot{\theta_1}*c\theta_4 + \dot{d_3}]])

In [19]:
#The Jacobian of velocities of the end effector in frame {e} is:
Jee = Matrix([[Vee[0].diff(theta1_dot), Vee[0].diff(d2_dot), Vee[0].diff(d3_dot), Vee[0].diff(theta4_dot)],
              [Vee[1].diff(theta1_dot), Vee[1].diff(d2_dot), Vee[1].diff(d3_dot), Vee[1].diff(theta4_dot)],
              [Vee[2].diff(theta1_dot), Vee[2].diff(d2_dot), Vee[2].diff(d3_dot), Vee[2].diff(theta4_dot)],
              [Wee[0].diff(theta1_dot), Wee[0].diff(d2_dot), Wee[0].diff(d3_dot), Wee[0].diff(theta4_dot)],
              [Wee[1].diff(theta1_dot), Wee[1].diff(d2_dot), Wee[1].diff(d3_dot), Wee[1].diff(theta4_dot)],
              [Wee[2].diff(theta1_dot), Wee[2].diff(d2_dot), Wee[2].diff(d3_dot), Wee[2].diff(theta4_dot)]])

symprint('J', 'e', 'e')
matprint(Jee, alias)

^eJ_e

Matrix([
[ c\theta_4*d_3 + 19*c\theta_4, s\theta_4, 0, 0],
[-d_3*s\theta_4 - 19*s\theta_4, c\theta_4, 0, 1],
[                   -c\theta_4,         0, 1, 0],
[                    s\theta_4,         0, 0, 0],
[                    c\theta_4,         0, 0, 0],
[                            0,         0, 0, 1]])

In [20]:
J0e = Matrix([[R0e, zeros(3)], [zeros(3), R0e]]) * Jee
symprint('J', 0, 'e')
matprint(J0e, alias)

print('After apply .simplify():')
J0e.simplify()
symprint('J', 0, 'e')
matprint(J0e, alias)

^0J_e

Matrix([
[c\theta_1*c\theta_4*(c\theta_4*d_3 + 19*c\theta_4) - c\theta_1*s\theta_4*(-d_3*s\theta_4 - 19*s\theta_4) - c\theta_4*s\theta_1,                           0,  s\theta_1, -c\theta_1*s\theta_4],
[c\theta_1*c\theta_4 + c\theta_4*s\theta_1*(c\theta_4*d_3 + 19*c\theta_4) - s\theta_1*s\theta_4*(-d_3*s\theta_4 - 19*s\theta_4),                           0, -c\theta_1, -s\theta_1*s\theta_4],
[                                          c\theta_4*(-d_3*s\theta_4 - 19*s\theta_4) + s\theta_4*(c\theta_4*d_3 + 19*c\theta_4), c\theta_4**2 + s\theta_4**2,          0,            c\theta_4],
[                                                                                                                             0,                           0,          0,            s\theta_1],
[                                                                                                                             0,                           0,          0,           -c\theta_1],
[                         

After apply .simplify():


^0J_e

Matrix([
[c\theta_1*d_3 + 19*c\theta_1 - c\theta_4*s\theta_1, 0,  s\theta_1, -c\theta_1*s\theta_4],
[c\theta_1*c\theta_4 + d_3*s\theta_1 + 19*s\theta_1, 0, -c\theta_1, -s\theta_1*s\theta_4],
[                                                 0, 1,          0,            c\theta_4],
[                                                 0, 0,          0,            s\theta_1],
[                                                 0, 0,          0,           -c\theta_1],
[                                                 1, 0,          0,                    0]])